In [ ]:
# installing required libraries
!pip install geopandas
!add-apt-repository ppa:ubuntugis/ppa -y
!apt-get update
!apt-get install python-numpy gdal-bin libgdal-dev python3-rtree
!pip install rasterio

Hit:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:4 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:7 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:9 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu bionic InRelease
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:11 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:13 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:14 https://develope

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torch.optim as optim
import os


%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import skimage
import matplotlib.pyplot as plt
import numpy as np


%cd /content/gdrive/MyDrive/SpaceNet
import libs.solaris as sol # modified solaris library
from utils.dist import *
from data.dataset import *
from models.unet import *
from glob import glob

Mounted at /content/gdrive
/content/gdrive/MyDrive/SpaceNet


In [ ]:
# load data and define transform
import torchvision
import torchvision.transforms as transforms

dataset = Spacenet()

train, test = torch.utils.data.random_split(dataset, [int(len(dataset)*0.8), len(dataset) - int(len(dataset)*0.8)])
train_loader = torch.utils.data.DataLoader(train, batch_size = 12, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test, batch_size= 12, shuffle=True, num_workers=2)

In [ ]:
# baseline model
device = 'cuda'

net = Unet(8, 2)
net = net.to(device)
#cudnn.benchmark = True

file_name = 'Unet_khartoum.pt'
trained_dir = '/content/gdrive/MyDrive/SpaceNet/trained'

criterion = nn.CrossEntropyLoss() 
optimizer = optim.Adam(net.parameters(), lr=0.0001, weight_decay = 0.1)

In [ ]:
# train and evaluate
from utils.eval import *
import math
val_loss = []

def train(epoch):
  print('\n Current train epoch : ', epoch)
  net.train()
  for batch_idx, (imgs, targets) in enumerate(train_loader):
    imgs, targets = imgs.to(device), targets.to(device)
    optimizer.zero_grad()
    outputs = net(imgs)

    loss = criterion(outputs, targets)
  
    loss.backward()
    if math.isnan(loss.item()):
      raise KeyboardInterrupt('nan detected')
    print('loss: ', loss.item())
    optimizer.step()

def test(epoch):
  print('\n current validation epoch: ', epoch)
  net.eval()
  total_loss = 0
  for batch_idx, (imgs, targets) in enumerate(validation_loader):
    imgs, targets = imgs.to(device), targets.to(device)
    optimizer.zero_grad()
    outputs = net(imgs)

    loss = criterion(outputs, targets)
    total_loss += loss.item()/10 # divide by batch size 

  state = {
      'net': net.state_dict(),
      'epoch': epoch,
      'optimizer': optimizer.state_dict()
  }
  torch.save(state, os.path.join(trained_dir, file_name))
  val_loss.append(total_loss)
  print('\n Current loss : ', total_loss) # get segmentation metrics 

In [ ]:
for epoch in range(1, 60):
  train(epoch)
  test(epoch)


 Current train epoch :  1


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


loss:  0.54930579662323
loss:  0.5653513073921204
loss:  0.5833499431610107
loss:  0.5538375973701477
loss:  0.5660769939422607
loss:  0.49234533309936523
loss:  0.5572530031204224
loss:  0.5570647120475769
loss:  0.505424976348877
loss:  0.5360609889030457
loss:  0.5063971281051636
loss:  0.46998292207717896
loss:  0.5005447268486023
loss:  0.4560466706752777
loss:  0.4538591504096985
loss:  0.4978424906730652
loss:  0.439436137676239
loss:  0.51175457239151
loss:  0.44723597168922424
loss:  0.47697848081588745
loss:  0.46969494223594666
loss:  0.4779142439365387
loss:  0.40419942140579224
loss:  0.4588490128517151
loss:  0.47676077485084534
loss:  0.4782065451145172
loss:  0.5145884156227112
loss:  0.489246666431427
loss:  0.4569537043571472
loss:  0.4522032141685486
loss:  0.4213392734527588
loss:  0.4633868634700775
loss:  0.477973073720932
loss:  0.47459450364112854
loss:  0.4388846457004547
loss:  0.449097216129303
loss:  0.4310566782951355
loss:  0.4600212574005127
loss:  0.4398

In [ ]:
# for data post processing
